In [1]:
import pandas as pd
import openai

prompts = pd.read_csv('fn1.7-test-prompts.csv')


In [3]:
from concurrent.futures import ThreadPoolExecutor
import re

def clean_output(output):
    clean_output = output
    # Extract XML code from the output
    xml_code = re.search(r'```xml(.*?)```', output, re.DOTALL)
    clean_output = ''
    if xml_code:
        clean_output = xml_code.group(1).strip()

    return clean_output

def prompt_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": prompt}
        ]
    )

    output = response.choices[0].message.content
    
    return clean_output(output)

def parallel_prompt_gpt(prompts):
    with ThreadPoolExecutor(max_workers=5) as executor:
        outputs = list(executor.map(prompt_gpt, prompts))

    return outputs

client = openai.Client(api_key='sk-proj-')

outputs = parallel_prompt_gpt(prompts['prompt'].values)

prompts['prediction'] = outputs

prompts.to_csv('fn1.7-test-gpt-4o-mini-tempurature-1-descriptions-p2.csv', index=False)